## 多次元配列
Juliaでは、**Array{T,N}という多次元配列が標準で用意されており、Tは多次元配列の要素の型、Nは次元数を示す。**
JuliaのArray型は**MATLABの多次元配列と似ており、関数名もMATLABと同じものになっているのが多い。**

#### pythonのNumpyとの違い
Numpyはインデックスが0から始まり、要素の順番はデフォルトでrow-major orderである。一方で、Juliaの多次元配列は、インデックスが1で始まり、要素の順番は**column-major order**である。

また、pythonのNumpyではarray型が多次元配列で、それとは別に線形代数演算のためにmatrix型というarray型のサブクラスが用意されている。一方で、Juliaの配列は全てArray{T,N}で統一されている。Vector{T}型とMatrix{T}型も存在するが、それぞれArray{T,1}, Array{T,2}の単なるエイリアスでしかない。

そして、Juliaはcolumn-major-orderであるため、行列の場合、**同じ列の要素を順番に列挙するのは極めて高速であるが、同じ行の要素を順番に列挙するのはあまり効率的ではない。**

また、前回のイントロでも紹介したように、Juliaでは慣用的に最後の関数名に!をつけたものは引数のすべてまたは一部を変更あるいは破棄することを示している。例えば、**sort関数は引数の配列を並び替えた新しい配列を返すが、sort!関数は引数の配列の中身を変更して並び替える。**

### 初期化

In [3]:
# 値が初期化されていない配列
println(Array{Float32}(undef, 3, 2))

# 0で初期化
println(zeros(Float32, 3, 2))

# 1で初期化
println(ones(Float32, 3, 2))

# 一様分布で初期化された配列
println(rand(Float32, 3, 2))

# 正規分布で初期化された配列
println(randn(Float32, 3, 2))

# 値を1.1で初期化
println(fill(1.1, 3, 2))

# 配列と類似した配列(値は初期化されていない)
A = rand(Float32, 3, 2)
println(similar(A))
println(similar(A, Float64))

# 配列のサイズを変更
A = rand(Float32, 3, 2)
println(reshape(A, 3, 2))

Float32[0.0 0.0; 0.0 0.0; 1.0f-45 0.0]
Float32[0.0 0.0; 0.0 0.0; 0.0 0.0]
Float32[1.0 1.0; 1.0 1.0; 1.0 1.0]
Float32[0.5152792 0.6615696; 0.29816592 0.38493788; 0.085704565 0.66170585]
Float32[2.3521376 0.055408146; 0.2969087 2.3363855; 1.7175344 1.3405255]
[1.1 1.1; 1.1 1.1; 1.1 1.1]
Float32[9.611649f-26 1.0f-45; 1.0f-45 4.1007285f-21; 3.394806f-20 1.0f-45]
[0.0 1.0e-323; 5.0e-324 5.0e-324; 2.372230349e-314 2.3722303495e-314]
Float32[0.77725506 0.33722687; 0.79548883 0.5556512; 0.15761924 0.6114229]


copy(A)は、配列Aをコピーするが、要素自体を再起的にコピーすることはできない。要素がInt型やFloat64型などのプリミティブ型であれば、copy(A)で十分。

ただし、中身が複合型で中身も含めて完全にコピーしたい場合はdeepcopy(A)を使用する。

### 基本的な操作

In [6]:
A = rand(Float32, 3, 2)

# 配列の要素の型
println(eltype(A))

# 配列の要素数
println(length(A))

# 配列の次元数
println(ndims(A))

# 配列のサイズ
println(size(A), size(A, 1))

# 配列のストライド
println(strides(A), stride(A, 2))

Float32
6
2
(3, 2)3
(1, 3)3


### インデクシング
配列の値を取得したり代入するための操作

In [9]:
A = collect(reshape(1:9, 3, 3))
println(A)

A[3, 3] = -9;

A[1:2, 1:2] = [-1 -4; -2 -5];

A

[1 4 7; 2 5 8; 3 6 9]


3×3 Array{Int64,2}:
 -1  -4   7
 -2  -5   8
  3   6  -9

### 多次元配列の演算
A+B, A-B, A*B, A/Bなどの演算が提供されているが、これがA.+B,A.-B,A.*Bになると要素同士の加算や乗算になる。

### ブロードキャスティング
ブロードキャスティングは、サイズが異なる配列同士の演算を効率的に実行する仕組みである。

In [13]:
# 非効率的なやり方
A = rand(Float32, 100, 100)
B = ones(Float32, 100, 100)
A+B;

# 効率的なやり方(numpyだと自動でブロードキャストしてくれてたが、juliaは明示的に行う必要性がある)
A = rand(Float32, 100, 100)
A .+ 1

100×100 Array{Float32,2}:
 1.66667  1.23236  1.42783  1.89919  …  1.40313  1.66356  1.30728  1.8893
 1.9107   1.78068  1.72072  1.60574     1.66235  1.00362  1.57327  1.30361
 1.55334  1.9741   1.86078  1.3759      1.47176  1.95632  1.80109  1.20698
 1.90323  1.55908  1.06174  1.23948     1.01975  1.84608  1.70055  1.72817
 1.39048  1.94447  1.10119  1.36478     1.27268  1.46819  1.16011  1.05644
 1.08242  1.64313  1.27739  1.25334  …  1.51736  1.58382  1.75486  1.18578
 1.33217  1.28345  1.71584  1.46547     1.45982  1.73028  1.47121  1.13048
 1.56387  1.70982  1.78385  1.75339     1.84996  1.20207  1.71935  1.24044
 1.76981  1.42385  1.85057  1.0488      1.05462  1.15056  1.92824  1.60645
 1.25484  1.6049   1.16261  1.72645     1.97218  1.91303  1.15041  1.34747
 1.26116  1.04424  1.69884  1.70067  …  1.99687  1.02356  1.87117  1.52424
 1.86628  1.84793  1.44121  1.07886     1.87287  1.83397  1.96833  1.15817
 1.5583   1.63687  1.43468  1.00855     1.64319  1.05892  1.58577  1.23311


In [14]:
# また、行列の各行に行ベクトルを加算する場合にもブロードキャスティングを適用できる
A = rand(Float32, 3, 3)

B = ones(Float32, 3, 3)

A.+ B

3×3 Array{Float32,2}:
 1.30396  1.55344  1.53307
 1.40673  1.34212  1.43025
 1.45862  1.14164  1.36936

上記の例では、Bのベクトルが自動的にAのサイズに拡張されて加算が実行されている。ちなみに、ブロードキャスティングできる条件は、

・配列同士の各次元の大きさが同じ

あるいは

・次元の大きさが異なる場合、片方のサイズが1

のどちらかのケースに限られる。また、ブロードキャスティングを行う関数としてbroadcast関数と broadcast!関数が用意されている。実際に、先程のA .+ Bと言う演算は、以下のbroadcastと同じ

In [15]:
# broadcast(f, args...)という構文になっており、argsを可変長引数として取っている。
broadcast(+, A, B)

3×3 Array{Float32,2}:
 1.30396  1.55344  1.53307
 1.40673  1.34212  1.43025
 1.45862  1.14164  1.36936

また、broadcastはf(args...)をf.(args...)と変更するだけで手軽にブロードキャストできる。また、argsは配列に限定されず、タプルなど他のコレクションにも適用可能である(**ドット演算**と呼ぶ)

In [22]:
sigmoid(x::Float64) = 1.0 / (1.0 + exp(-x));
println(sigmoid(0.0))
println(sigmoid(1.0))

# sigmoidがFloat64なので、Float64で宣言しないとエラーになるので注意。型が定義されてなければ自動で対応する
A = rand(Float64, 3, 2)
sigmoid.(A)

0.5
0.7310585786300049


3×2 Array{Float64,2}:
 0.517282  0.581058
 0.652537  0.605759
 0.636362  0.507583

別の方法としてmap(sigmoid, A)とする方法もあるが、簡潔な場合はドット演算を使った方が良い

### map, reduce, filter

配列の各要素に関数を適用して値を変換したり、集約したり、フィルタリングするにはmap, reduce, filterなどの関数を使用するのが便利

map: コレクションの要素を変換する関数

reduce:コレクションの要素を集約する関数

filter:コレクションの要素に対する条件評価を行うための関数

In [23]:
A = rand(3, 2)
map(x -> x + 1.0, A)

# 全ての要素の積
reduce(*, A)

filter(x -> x < 0.5, A)

5-element Array{Float64,1}:
 0.1850654834826575
 0.14926308983701952
 0.464651661631553
 0.17666649462980089
 0.005888566740722467

### サブ配列
配列の一部を表すためのオブジェクト。view関数を使って、配列からサブ配列を作成できる

In [24]:
A = rand(3, 3)

view(A, 1, 2:3)

2-element view(::Array{Float64,2}, 1, 2:3) with eltype Float64:
 0.9880904283482446
 0.8703247014037034

In [26]:
view(A, 1:2, :)

2×3 view(::Array{Float64,2}, 1:2, :) with eltype Float64:
 0.685776  0.98809   0.870325
 0.499433  0.590099  0.581659

Aから作成したサブ配列は、**Aへの参照とインデックスの情報を保持しており、値を直接保持しているわけではない。**
よって、Aの中身が変更されると、それに応じてサブ配列の値も変更される。

サブ配列の利点は参照情報しか持たないため、巨大な配列から一部を取り出すような状況でも非常に効率的であるという点である。また、Juliaの配列に関する多くの関数はサブ配列にも対応しているため、巨大な配列の一部を一時的に取り出して計算を行うときはサブ配列の方が良い(しかし、巨大な配列でない限りはインデクシングの方が早いことも多い)

## モジュール

ソースコードが大きくなってくると、意図せず同じ名前を別々のものにつけてしまう現象はよくある。このような事故を**名前の衝突**という。これらの現象は**名前空間を使用することで回避することができる。**

名前空間とは、名前(識別子)を分割して管理するための空間であり、同じ名前であっても名前空間が別ならば、衝突せずに共存することができる。パッケージの利用や開発する上でも、名前空間を理解することは非常に重要

あるモジュールの名前空間に属する名前は、そのモジュール名を通して参照することができる。例えば、Aというモジュールの中にfooという名前の関数があったとする。Aモジュールが定める名前空間の外ではfoo関数を直接参照できないが、A.fooと書くことでモジュール名を通して間接的に参照するkとができる。

Juliaのコードは、常にモジュールに関連ついた名前空間の中で実行されている。例えばREPLを起動してgcd(12,8)と入力すると12と8の最大公約数が計算できるが、このときJuliaはMainと呼ばれるデフォルトで存在するモジュールの名前空間からgcdの名前に対応する関数を探し出している。**コードの現在位置でどのモジュールが有効になっているかは、@\_\_MODULE\_\_マクロで確認できる**

In [34]:
@__MODULE__

Main

現在のモジュールの名前空間に名前を追加するには二つの方法がある。一つ目の方法は、**現在のモジュールで新しい名前を定義する方法**。二つ目の方法は、**既にある他のモジュールから現在のモジュールに名前を取り込む方法**である。

### 既存モジュールの利用
この方法は、Juliaの標準ライブラリやパッケージを利用するときの方法である。

Juliaの標準ライブラリにあるStatisticsモジュールを例にする(meanやstdなどの関数が存在する)。Statisticsモジュールから提供されている関数名を現在のモジュールに取り込むには、**using文を使うのが一般的である**。using Statisticsのように、対象のモジュール名を指定して呼び出す。

また、**usingで名前を指定せずに取り込むことは可能だが、意図しない名前も取り込んでしまうので、ソースコードを読みやすくするには取り込む名前も指定してあげた方が良い**。

注意：usingと似たような機能でimport文がある。こちらはusingと異なり、モジュールから暗黙的に名前を取り込まない。

In [28]:
#初めはmean関数は存在しない
#mean([1,2,3])

using Statistics
mean([1,2,3])

2.0

In [30]:
# :の後に取り込む名前の指定もできる複数取り込むときは,でつなげる
using Statistics: mean, std
mean([1,2,3]) # OK
#std([1,2,3]) # NG

2.0

In [31]:
# usingはモジュール自身も取り込める
using Statistics: Statistics

Statistics.mean([1,2,3])

# mean([1,2,3]) # エラーになる

2.0

### 新しいモジュールの定義
新しいモジュールを定義するときは、moduleキーワードを用いる。モジュールの名前は慣習として大文字から始める。また、慣習としてmoduleとendの間はインデントを行わない。

moduleからendの間に書かれたソースコードで定義される名前は、現在定義しているモジュールの名前空間に属する。したがって、それ以前に同名の関数やグローバル変数を定義していたとしても、上書きされることはない。

In [32]:
module Greeting
hello(name) = println("Hello, $(name).")
end

Main.Greeting

In [33]:
Greeting.hello("Julia")

Hello, Julia.


前述では、using文を使って他のモジュールから現在のモジュールに名前を取り込めることを説明したが、このときどの名前が取り込まれるかはモジュール定義のときに指定する。**export文はモジュール定義の中で行い、そのモジュールがusing文で読み込まれたときに取り込む名前を定義する。例えが、using文を実行したときにfoo関数を取り込むようにしたければexport fooとモジュール定義内に記述する**。

using Statisticsを実行するとmeanやstdなどの関数名が現在のモジュールに取り込まれていたが、それらもexport文を使ってモジュールからエクスポートするように指定されている。以下にexport文を使ったモジュールの定義の例を示す。こちらの例ではgoodbye関数も定義されているが、こちらの名前はエクスポートするように指定されていない。

しかし、**Greeting.goodbyeのようにモジュール名を経由することでエクスポートされていない名前も取得することができる**。

In [35]:
module Greeting
export hello
hello(name) = println("Hello, $(name).")
goodbye(name) = println("Goodbye, $(name).")
end

Main.Greeting

また、モジュールの構造は入れ子にもできる。次のコード例では、Aモジュールの中でB1とB2モジュールを定義している。モジュールの定義では通常インデントをつけないが、今回は見やすさを重視するためにインデントを加えている。

子のモジュールは親のモジュールを経由することで参照することができる。

### モジュールの相対パス指定

using文やimport文で指定されたモジュール名は、基本的にLOAD_PATHという変数に収められたプロジェクトのパスから検索される。

module文を使って定義した直後のモジュールは、using文を使って読み込もうとしても失敗する。これは、using GreetingがLOAD_PATHからGreetingモジュールを探し出そうとするためである。

よって、.ModuleNameは自分の子モジュールを指定する相対パスである。.GreetingでMainモジュールの子モジュールであるGreetingモジュールが指定されている。これよりも祖先の子モジュールを相対パスで指定するためには、ドットを必要な分だけ重ねる(..ModuleName, ...ModuleName)

In [37]:
# using Greeting(失敗)
using .Greeting # 相対パスによる読み込み
hello("Julia")

Hello, Julia.


以下に、ネストしたモジュールの相対パスの指定方法を示す

### ファイルの分割
モジュールを定義すると、そのソースコードが非常に長くなることがある。そうなると管理が難しくなるため、適当な単位で分割した管理したい時がある。**Juliaではinclude関数を使ってモジュールを複数のファイルに分けて定義することができる**。下記の例では、三つのファイルから構成されるモジュールを定義できる。

ファイルの取り込みには絶対パスも使用できるが、**通常ソースコードのファイルはプロジェクト単位で管理されるので、絶対パスを使うとプロジェクトの移動により正しくファイルが読み込めなくなる恐れがある**。従って、**include関数には、その関数を呼び出したファイルを収めたディレクトリからの相対パスを指定するのが一般的である**。

### 他のモジュールで定義された関数の拡張
Juliaは多重ディスパッチを中心としたプログラミング言語であり、あるモジュールで定義された関数を外部から拡張する機能を持っている。
**using文では取り込んだ関数の拡張ができないが、import文では拡張できる**という違いがある。

次の例では、ユーザが定義したVec3型に対してlength関数を呼び出せるように拡張している。**import文を使ってlength関数をBaseモジュールから取り込むことで、長さを計算する操作をVec3型にも拡張することができる**。

In [39]:
# 長さが3に固定されたベクトルの定義(抽象型のパラメトリック型を使用している)
struct Vec3{T} <: AbstractVector{T}
    x::T
    y::T
    z::T
end

# length関数をBaseモジュールから拡張するために取り組む
import Base: length
length(v::Vec3) = 3

length (generic function with 85 methods)